<a href="https://colab.research.google.com/github/kejeon/in_dev_RN20Q/blob/main/2_sweep_resent20_q_with_KD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL = not IN_COLAB
USE_GITHUB = True
USE_DRIVE = False
device = 'cuda'

assert not (USE_GITHUB and USE_DRIVE)

In [ ]:
if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  if USE_GITHUB:
    !git clone https://github.com/kejeon/in_dev_RN20Q.git
    %cd '/content/in_dev_RN20Q'
  elif USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
Cloning into 'in_dev_RN20Q'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 114 (delta 55), reused 81 (delta 31), pack-reused 0
Receiving objects: 100% (114/114), 2.04 MiB | 4.50 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/in_dev_RN20Q


In [ ]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')
state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device(device))

teacher_model = resnet20()

try:
  teacher_model.load_state_dict(state_dict['net'])
except:
  # if the model is wrapped in a module, update all keys in state_dict to remove module.
  state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}
  teacher_model.load_state_dict(state_dict['net'])

In [ ]:
def train():
    run = wandb.init()

    arch_tag = "ResNet20_Q"
    batch_size = 128
    dataset = "CIFAR10"
    T=wandb.config.T
    alpha=wandb.config.alpha
    lr=0.012

    student_model = ResNet20_Q(a_bit=4, w_bit=1)

    my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           T=T,
                           alpha=alpha,
                           device=device,
                           batch_size=batch_size,
                           lr=lr)

    my_trainer.train_script(200)

    return

In [ ]:
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'test_acc'},
    'parameters':
    {
        'T': {'values': [4,8,12,16,20]},
        'alpha': {'values': [1, 0.8, 0.6, 0.4, 0.2, 0]}
     }
}

sweep_id = wandb.sweep(
  sweep=sweep_configuration,
  project='my-first-sweep'
  )

In [ ]:
wandb.agent(sweep_id, function=train, count=5)